In [1]:
from bs4 import BeautifulSoup as bs
import requests

# Step 1: Get movie info for Iron Man(2008 film)

In [2]:
r = requests.get("https://en.wikipedia.org/wiki/Iron_Man_(2008_film)")

# Convert to a beautiful soup object
soup = bs(r.content)

# Print out the HTML
contents = soup.prettify()
#print(contents)

In [3]:
info = soup.find(class_="infobox vevent")
info_rows = info.find_all("tr")
#for row in info_rows:
   #print(row.prettify())

In [4]:
def get_content_value(row_data):
    if row_data.find("li"):
        return [li.get_text(" ", strip=True).replace("\xa0", " ") for li in row_data.find_all("li")]
    else:
        return row_data.get_text(" ", strip=True).replace("\xa0", " ")

In [5]:
movie_info = {}

for index, row in enumerate(info_rows):
    if index == 0:
        movie_info['title'] = row.find("th").get_text(" ", strip=True)
    elif index == 1:
        continue
    else:
        content_key = row.find("th").get_text(" ", strip=True)
        content_value = get_content_value(row.find("td"))
        movie_info[content_key] = content_value
    
movie_info

{'title': 'Iron Man',
 'Directed by': 'Jon Favreau',
 'Screenplay by': ['Mark Fergus Hawk Ostby', 'Art Marcum Matt Holloway'],
 'Based on': ['Stan Lee', 'Larry Lieber', 'Don Heck', 'Jack Kirby'],
 'Produced by': ['Avi Arad', 'Kevin Feige'],
 'Starring': ['Robert Downey Jr.',
  'Terrence Howard',
  'Jeff Bridges',
  'Shaun Toub',
  'Gwyneth Paltrow'],
 'Cinematography': 'Matthew Libatique',
 'Edited by': 'Dan Lebental',
 'Music by': 'Ramin Djawadi',
 'Production company': 'Marvel Studios',
 'Distributed by': 'Paramount Pictures [N 1]',
 'Release dates': ['April 14, 2008 ( 2008-04-14 ) (Sydney)',
  'May 2, 2008 ( 2008-05-02 ) (United States)'],
 'Running time': '126 minutes [4]',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$140 million [5]',
 'Box office': '$585.8 million [5]'}

# Step 2: Get movie info for all movies

In [6]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_Marvel_Cinematic_Universe_films")

In [7]:
# Convert to a beautiful soup object
soup = bs(r.content)

# Print out the HTML
contents = soup.prettify()
#print(contents)all

In [8]:
movies = soup.select(".wikitable.sortable i")
movies

[<i><a href="/wiki/Iron_Man_(2008_film)#Box_office" title="Iron Man (2008 film)">Iron Man</a></i>,
 <i><a href="/wiki/The_Incredible_Hulk_(film)#Box_office" title="The Incredible Hulk (film)">The Incredible Hulk</a></i>,
 <i><a href="/wiki/Iron_Man_2#Box_office" title="Iron Man 2">Iron Man 2</a></i>,
 <i><a href="/wiki/Thor_(film)#Box_office" title="Thor (film)">Thor</a></i>,
 <i><a href="/wiki/Captain_America:_The_First_Avenger#Box_office" title="Captain America: The First Avenger">Captain America: The First Avenger</a></i>,
 <i><a href="/wiki/The_Avengers_(2012_film)#Box_office" title="The Avengers (2012 film)">Marvel's The Avengers</a></i>,
 <i><a href="/wiki/Iron_Man_3#Box_office" title="Iron Man 3">Iron Man 3</a></i>,
 <i><a href="/wiki/Thor:_The_Dark_World#Box_office" title="Thor: The Dark World">Thor: The Dark World</a></i>,
 <i><a href="/wiki/Captain_America:_The_Winter_Soldier#Box_office" title="Captain America: The Winter Soldier">Captain America: The Winter Soldier</a></i>,


#### Somehow movies are dupilcated, we only need the half of list.

In [9]:
movies = movies[0:int((len(movies)/2))]

#### Here, I want to created functions to do our step one.

In [10]:
def get_content_value(row_data):
    if row_data.find("li"):
        return [li.get_text(" ", strip=True).replace("\xa0", " ") for li in row_data.find_all("li")]
    elif row_data.find("br"):
        return [text for text in row_data.stripped_strings]
    else:
        return row_data.get_text(" ", strip=True).replace("\xa0", " ")

def clean_tags(soup):
    for tag in soup.find_all(["sup", "span"]):
        tag.decompose()
        
def get_info(url):

    r = requests.get(url)
    soup = bs(r.content)
    info_box = soup.find(class_="infobox vevent")
    info_rows = info_box.find_all("tr")
    
    clean_tags(soup)

    movie_info = {}
    for index, row in enumerate(info_rows):
        if index == 0:
            movie_info['title'] = row.find("th").get_text(" ", strip=True)
        else:
            header = row.find('th')
            if header:
                content_key = row.find("th").get_text(" ", strip=True)
                content_value = get_content_value(row.find("td"))
                movie_info[content_key] = content_value
            
    return movie_info    

#### Try function on the movie 'The Incredible Hulk'

In [11]:
get_info("https://en.wikipedia.org/wiki/The_Incredible_Hulk_(film)")

{'title': 'The Incredible Hulk',
 'Directed by': 'Louis Leterrier',
 'Written by': 'Zak Penn',
 'Based on': ['Stan Lee', 'Jack Kirby'],
 'Produced by': ['Avi Arad', 'Gale Anne Hurd', 'Kevin Feige'],
 'Starring': ['Edward Norton',
  'Liv Tyler',
  'Tim Roth',
  'Tim Blake Nelson',
  'Ty Burrell',
  'William Hurt'],
 'Cinematography': 'Peter Menzies Jr.',
 'Edited by': ['John Wright', 'Rick Shaine', 'Vincent Tabaillon'],
 'Music by': 'Craig Armstrong',
 'Production companies': ['Marvel Studios', 'Valhalla Motion Pictures'],
 'Distributed by': 'Universal Pictures',
 'Release dates': ['June 8, 2008 ( Gibson Amphitheatre )',
  'June 13, 2008 (United States)'],
 'Running time': '112 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$137.5–150 million',
 'Box office': '$264.8 million'}

#### Since the url of each movie is in certain format, we use a for loop here to use function get_info on each movie.

In [12]:
r = requests.get("https://en.wikipedia.org/wiki/List_of_Marvel_Cinematic_Universe_films")
soup = bs(r.content)
movies = soup.select(".wikitable.sortable i a")

base_path = "https://en.wikipedia.org/"

movie_info_list = []
for index, movie in enumerate(movies):
    if index % 10 == 0:
        print(index)
    try:
        relative_path = movie['href']
        full_path = base_path + relative_path
        title = movie['title']
        
        movie_info_list.append(get_info(full_path))
        
    except Exception as e:
        print(movie.get_text())
        print(e)

0
10
20
30
40
50


In [13]:
movie_info_list = movie_info_list[0:int((len(movies)/2))]

In [15]:
movie_info_list[0]

{'title': 'Iron Man',
 'Directed by': 'Jon Favreau',
 'Screenplay by': ['Mark Fergus Hawk Ostby', 'Art Marcum Matt Holloway'],
 'Based on': ['Stan Lee', 'Larry Lieber', 'Don Heck', 'Jack Kirby'],
 'Produced by': ['Avi Arad', 'Kevin Feige'],
 'Starring': ['Robert Downey Jr.',
  'Terrence Howard',
  'Jeff Bridges',
  'Shaun Toub',
  'Gwyneth Paltrow'],
 'Cinematography': 'Matthew Libatique',
 'Edited by': 'Dan Lebental',
 'Music by': 'Ramin Djawadi',
 'Production company': 'Marvel Studios',
 'Distributed by': 'Paramount Pictures',
 'Release dates': ['April 14, 2008 (Sydney)', 'May 2, 2008 (United States)'],
 'Running time': '126 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$140 million',
 'Box office': '$585.8 million'}

# Step 3: Some data cleaning 

#### Change the "running time" data type to integer.

In [16]:
clean_list = movie_info_list

In [17]:
def minutes_to_integer(running_time):
    if running_time == "N/A":
        return None
    
    if isinstance(running_time, list):
        return int(running_time[0].split(" ")[0])
    else: # is a string
        return int(running_time.split(" ")[0])

for movie in clean_list:
    movie['Running time'] = minutes_to_integer(movie.get('Running time', "N/A"))

In [18]:
clean_list[0]

{'title': 'Iron Man',
 'Directed by': 'Jon Favreau',
 'Screenplay by': ['Mark Fergus Hawk Ostby', 'Art Marcum Matt Holloway'],
 'Based on': ['Stan Lee', 'Larry Lieber', 'Don Heck', 'Jack Kirby'],
 'Produced by': ['Avi Arad', 'Kevin Feige'],
 'Starring': ['Robert Downey Jr.',
  'Terrence Howard',
  'Jeff Bridges',
  'Shaun Toub',
  'Gwyneth Paltrow'],
 'Cinematography': 'Matthew Libatique',
 'Edited by': 'Dan Lebental',
 'Music by': 'Ramin Djawadi',
 'Production company': 'Marvel Studios',
 'Distributed by': 'Paramount Pictures',
 'Release dates': ['April 14, 2008 (Sydney)', 'May 2, 2008 (United States)'],
 'Running time': 126,
 'Country': 'United States',
 'Language': 'English',
 'Budget': '$140 million',
 'Box office': '$585.8 million'}

#### Change the 'Budget' and 'Box office' data types to integer.

In [19]:
#import functions to convert money from string to float from money.py
from money import *

In [20]:
for movie in clean_list:
    movie['Budget'] = int(money_conversion(movie.get('Budget', "N/A")))
    movie['Box office'] = int(money_conversion(movie.get('Box office', "N/A")))

In [21]:
#money_conversion(str(movie_info_list[-40]["Budget"]))
clean_list[0]

{'title': 'Iron Man',
 'Directed by': 'Jon Favreau',
 'Screenplay by': ['Mark Fergus Hawk Ostby', 'Art Marcum Matt Holloway'],
 'Based on': ['Stan Lee', 'Larry Lieber', 'Don Heck', 'Jack Kirby'],
 'Produced by': ['Avi Arad', 'Kevin Feige'],
 'Starring': ['Robert Downey Jr.',
  'Terrence Howard',
  'Jeff Bridges',
  'Shaun Toub',
  'Gwyneth Paltrow'],
 'Cinematography': 'Matthew Libatique',
 'Edited by': 'Dan Lebental',
 'Music by': 'Ramin Djawadi',
 'Production company': 'Marvel Studios',
 'Distributed by': 'Paramount Pictures',
 'Release dates': ['April 14, 2008 (Sydney)', 'May 2, 2008 (United States)'],
 'Running time': 126,
 'Country': 'United States',
 'Language': 'English',
 'Budget': 140000000,
 'Box office': 585800000}

# Step 4: save data as JSON and load
#### JSON file can be a good choice of non-relational database

In [29]:
import json

def save_data(title, data):
    with open(title, 'w', encoding='utf-8') as f:
        json.dump(data, f, ensure_ascii=False, indent=2)

save_data("marvel_movie.json", clean_list)

In [36]:
from IPython.display import JSON

def load_data(title):
    with open(title, encoding="utf-8") as f:
        return JSON(json.load(f))
    
load_data('marvel_movie.json')

<IPython.core.display.JSON object>

# Step 5: convert DataFrame to SQL database

In [22]:
import pandas as pd
import sqlite3

df = pd.DataFrame(clean_list)
df.head(3)
df = df.applymap(str)

In [23]:
cnn = sqlite3.connect('movie.db')
df.to_sql('movie', cnn)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\generic.py:2882: UserWarning: The spaces in these column names will not be changed. In pandas versions < 0.14, spaces were converted to underscores.
  method=method,


In [24]:
%load_ext sql
%sql sqlite:///movie.db

#### Let's run some sql query on our database.

#### Who starred the movie "Iron Man"? 'Robert Downey Jr.', 'Terrence Howard', 'Jeff Bridges', 'Shaun Toub', 'Gwyneth Paltrow'

In [47]:
%%sql
SELECT Starring
FROM movie
WHERE title = "Iron Man"

 * sqlite:///movie.db
Done.


Starring
"['Robert Downey Jr.', 'Terrence Howard', 'Jeff Bridges', 'Shaun Toub', 'Gwyneth Paltrow']"


#### Which one is the longest marvel movie so far? Avengers: Endgame.

In [60]:
%%sql
SELECT title, max("Running time")
FROM movie

 * sqlite:///movie.db
Done.


title,"max(""Running time"")"
Avengers: Endgame,181


#### Which three movies have the highest profit (Box office - Budget)? 'Avengers: Endgame', 'Avengers: Infinity War', 'Spider-Man: No Way Home'.

In [109]:
%%sql
SELECT title, ("Box office" - Budget) AS Profit
FROM movie
ORDER BY ("Box office" - Budget) DESC

 * sqlite:///movie.db
Done.


title,Profit
Avengers: Endgame,2442000000
Avengers: Infinity War,1723000000
Spider-Man: No Way Home,1716000000
The Avengers,1299000000
Black Panther,1148000000
Iron Man 3,1015000000
Captain Marvel,976000000
Spider-Man: Far From Home,972000000
Avengers: Age of Ultron,959000000
Captain America: Civil War,903000000


#### What's the total box office sales for Iron Man series? 2424700000, which is 2.4 billions.

In [112]:
%%sql
SELECT SUM("Box office")
FROM movie
WHERE title LIKE 'Iron Man%'

 * sqlite:///movie.db
Done.


"SUM(""Box office"")"
2424700000
